In [1]:
import numpy as np
import pandas as pd
import os
import json
pd.set_option('max_rows', 5000)

In [2]:
# Read in the data CSV files
train = pd.read_csv('../input/data-science-bowl-2019/train.csv')
train_labels = pd.read_csv('../input/data-science-bowl-2019/train_labels.csv')
test = pd.read_csv('../input/data-science-bowl-2019/test.csv')
#specs = pd.read_csv('../input/data-science-bowl-2019/specs.csv')
#ss = pd.read_csv('../input/data-science-bowl-2019/sample_submission.csv')

In [3]:
keep_id = train[train.type == "Assessment"][['installation_id']].drop_duplicates()
train = pd.merge(train, keep_id, on="installation_id", how="inner")
train = train[train.installation_id.isin(train_labels.installation_id.unique())]
train.shape

(7734558, 11)

# title of games/videos when the type is assessment
it always includes (Assessment) in the title

In [4]:
train[train.type=="Assessment"]["title"].value_counts()

Mushroom Sorter (Assessment)    192906
Bird Measurer (Assessment)      188466
Chest Sorter (Assessment)       186421
Cauldron Filler (Assessment)    177669
Cart Balancer (Assessment)      158143
Name: title, dtype: int64

# finding match between train and train_labels by one id

In [5]:
summarized_df = train_labels[train_labels.installation_id == "0006a69f"]
#summarized_df.to_csv("summarized_result.csv", index=False)
summarized_df

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3


From Kaggle: The file train_labels.csv has been provided to show how these groups would be computed on the assessments in the training set. Assessment attempts are captured in event_code 4100 for all assessments except for Bird Measurer, which uses event_code 4110. If the attempt was correct, it contains "correct":true.

However, in the first version I already noticed that I had one attempt too many for this installation_id when mapping the rows with the train_labels for. It turns out that there are in fact also assessment attemps for Bird Measurer with event_code 4100, which should not count (see below). In this case that also makes sense as this installation_id already had a pass on the first attempt

In [6]:
#train[(train.installation_id == "0006a69f") & ((train.type == "Assessment") & (train.title == 'Bird Measurer (Assessment)') & (train.event_code == 4110) |
#                                               (train.type == "Assessment") & (train.title == 'Bird Measurer (Assessment)') & (train.event_code == 4100))]

In [7]:
train[(train.installation_id == "0006a69f") & ((train.type == "Assessment") & (train.title == 'Bird Measurer (Assessment)') & (train.event_code == 4110) |
                                               (train.type == "Assessment") & (train.title == 'Bird Measurer (Assessment)') & (train.event_code == 4100))].shape[0]

13

When we exclude the Bird Measurer/4100 rows we get the correct match with the numbers in train_labels for this installation_id (1 correct, 11 incorrect)

In [8]:
#train[(train.installation_id == "0006a69f") & (train.type == "Assessment") & (train.title == 'Bird Measurer (Assessment)') & (train.event_code == 4110)]

In [9]:
train[(train.installation_id == "0006a69f") & (train.type == "Assessment") & (train.title == 'Bird Measurer (Assessment)') & (train.event_code == 4110)].shape[0]

12

in terms of Mushroom Sorter (Assessment), there's no Mushromm Sorter/4110 row.

In [10]:
#train[(train.installation_id == "0006a69f") &  (train.type == "Assessment") & (train.title == 'Mushroom Sorter (Assessment)') & (train.event_code == 4100)]

In [11]:
assessment_df = train[(train.installation_id == "0006a69f") & ((train.type == "Assessment") & (train.title == 'Bird Measurer (Assessment)') & (train.event_code == 4110) |
                                               (train.type == "Assessment") & (train.title != 'Bird Measurer (Assessment)') & (train.event_code == 4100))]
#assessment_df.to_csv("assessment.csv", index=False)
assessment_df

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
690,25fa8af4,901acc108f55a5a1,2019-08-06T05:22:32.357Z,"{""correct"":true,""stumps"":[1,2,4],""event_count""...",0006a69f,44,4100,31011,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
1171,17113b36,77b8ee947eb84b4e,2019-08-06T05:35:54.898Z,"{""correct"":false,""caterpillars"":[11,8,3],""even...",0006a69f,29,4110,35771,Bird Measurer (Assessment),Assessment,TREETOPCITY
1177,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:01.927Z,"{""correct"":false,""caterpillars"":[11,8,11],""eve...",0006a69f,35,4110,42805,Bird Measurer (Assessment),Assessment,TREETOPCITY
1182,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:06.512Z,"{""correct"":false,""caterpillars"":[11,8,5],""even...",0006a69f,40,4110,47388,Bird Measurer (Assessment),Assessment,TREETOPCITY
1187,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:09.739Z,"{""correct"":false,""caterpillars"":[11,8,7],""even...",0006a69f,45,4110,50605,Bird Measurer (Assessment),Assessment,TREETOPCITY
1192,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:13.951Z,"{""correct"":false,""caterpillars"":[11,8,4],""even...",0006a69f,50,4110,54822,Bird Measurer (Assessment),Assessment,TREETOPCITY
1195,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:17.407Z,"{""correct"":false,""caterpillars"":[11,8,4],""even...",0006a69f,53,4110,58280,Bird Measurer (Assessment),Assessment,TREETOPCITY
1200,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:21.390Z,"{""correct"":false,""caterpillars"":[11,8,2],""even...",0006a69f,58,4110,62256,Bird Measurer (Assessment),Assessment,TREETOPCITY
1205,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:26.296Z,"{""correct"":false,""caterpillars"":[11,8,1],""even...",0006a69f,63,4110,67164,Bird Measurer (Assessment),Assessment,TREETOPCITY
1212,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:32.187Z,"{""correct"":false,""caterpillars"":[11,8,1],""even...",0006a69f,70,4110,73056,Bird Measurer (Assessment),Assessment,TREETOPCITY


In [12]:
all_data = train[train.installation_id == "0006a69f"]
#all_data.to_csv("all_data.csv", index=False)
all_data

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,34ba1a28d02ba8ba,2019-08-06T04:57:18.904Z,"{""event_code"": 2000, ""event_count"": 1}",0006a69f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,4b57c9a59474a1b9,2019-08-06T04:57:45.301Z,"{""event_code"": 2000, ""event_count"": 1}",0006a69f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,2b9d5af79bcdb79f,2019-08-06T04:58:14.538Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,2b9d5af79bcdb79f,2019-08-06T04:58:14.615Z,"{""description"":""Let's build a sandcastle! Firs...",0006a69f,2,3010,29,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1325467d,2b9d5af79bcdb79f,2019-08-06T04:58:16.680Z,"{""coordinates"":{""x"":273,""y"":650,""stage_width"":...",0006a69f,3,4070,2137,Sandcastle Builder (Activity),Activity,MAGMAPEAK
5,1325467d,2b9d5af79bcdb79f,2019-08-06T04:58:18.474Z,"{""coordinates"":{""x"":863,""y"":237,""stage_width"":...",0006a69f,4,4070,3937,Sandcastle Builder (Activity),Activity,MAGMAPEAK
6,1325467d,2b9d5af79bcdb79f,2019-08-06T04:58:19.365Z,"{""coordinates"":{""x"":817,""y"":617,""stage_width"":...",0006a69f,5,4070,4820,Sandcastle Builder (Activity),Activity,MAGMAPEAK
7,1bb5fbdb,2b9d5af79bcdb79f,2019-08-06T04:58:21.490Z,"{""description"":""Let's build a sandcastle! Firs...",0006a69f,6,3110,6954,Sandcastle Builder (Activity),Activity,MAGMAPEAK
8,1325467d,2b9d5af79bcdb79f,2019-08-06T04:58:22.732Z,"{""coordinates"":{""x"":809,""y"":180,""stage_width"":...",0006a69f,7,4070,8187,Sandcastle Builder (Activity),Activity,MAGMAPEAK
9,5e812b27,2b9d5af79bcdb79f,2019-08-06T04:58:23.295Z,"{""size"":0,""coordinates"":{""x"":782,""y"":207,""stag...",0006a69f,8,4030,8745,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [13]:
# list of index after assessment, which don't have to do with assessments
#remove_id = []
#for i in train_labels.installation_id.unique():
#    print("id: ", i)
#    game_sesion_in_id = train_labels[train_labels.installation_id == str(i)]["game_session"]
#    tmp_indexes = [train[train.game_session == str(i)].index[-1] for i in game_sesion_in_id]
#    remove_first = max(tmp_indexes)+1
#    remove_last = train[train.installation_id == str(i)].index[-1]
#    print(remove_first)
#    print(remove_last)
#    for j in range(remove_first, remove_last+1):
#        remove_id.append(j)

In [14]:
# game_session and title is connected with one by one.
# all_data.groupby("game_session")["title"].value_counts()

In [15]:
# game_session and type is connected with one by one.
# all_data.groupby("game_session")["type"].value_counts()

In [16]:
# game_session and world is connected with one by one
# all_data.groupby("game_session")["world"].value_counts()

In [17]:
all_data[all_data["event_data"].str.contains("correct")]

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
113,5c3d2b2f,0336db4dee65ad4b,2019-08-06T05:01:25.191Z,"{""size"":1,""item_type"":""Tub"",""position"":1,""anim...",0006a69f,9,4020,11886,Scrub-A-Dub,Game,MAGMAPEAK
118,5c3d2b2f,0336db4dee65ad4b,2019-08-06T05:01:26.585Z,"{""size"":4,""item_type"":""Tub"",""position"":2,""anim...",0006a69f,14,4020,13286,Scrub-A-Dub,Game,MAGMAPEAK
127,5c3d2b2f,0336db4dee65ad4b,2019-08-06T05:01:36.930Z,"{""size"":4,""item_type"":""Water"",""position"":2,""an...",0006a69f,23,4020,23628,Scrub-A-Dub,Game,MAGMAPEAK
130,5c3d2b2f,0336db4dee65ad4b,2019-08-06T05:01:38.438Z,"{""size"":1,""item_type"":""Water"",""position"":1,""an...",0006a69f,26,4020,25137,Scrub-A-Dub,Game,MAGMAPEAK
135,5c3d2b2f,0336db4dee65ad4b,2019-08-06T05:01:39.729Z,"{""size"":4,""item_type"":""Water"",""position"":2,""an...",0006a69f,31,4020,26433,Scrub-A-Dub,Game,MAGMAPEAK
145,5c3d2b2f,0336db4dee65ad4b,2019-08-06T05:01:50.131Z,"{""size"":1,""item_type"":""Soap"",""position"":1,""ani...",0006a69f,41,4020,36837,Scrub-A-Dub,Game,MAGMAPEAK
151,5c3d2b2f,0336db4dee65ad4b,2019-08-06T05:01:51.483Z,"{""size"":4,""item_type"":""Soap"",""position"":2,""ani...",0006a69f,47,4020,38187,Scrub-A-Dub,Game,MAGMAPEAK
164,5c3d2b2f,0336db4dee65ad4b,2019-08-06T05:02:22.188Z,"{""size"":3,""item_type"":""Tub"",""position"":1,""anim...",0006a69f,60,4020,68889,Scrub-A-Dub,Game,MAGMAPEAK
167,5c3d2b2f,0336db4dee65ad4b,2019-08-06T05:02:24.286Z,"{""size"":4,""item_type"":""Tub"",""position"":3,""anim...",0006a69f,63,4020,70989,Scrub-A-Dub,Game,MAGMAPEAK
172,5c3d2b2f,0336db4dee65ad4b,2019-08-06T05:02:25.862Z,"{""size"":3,""item_type"":""Tub"",""position"":1,""anim...",0006a69f,68,4020,72572,Scrub-A-Dub,Game,MAGMAPEAK


In [18]:
print("no of irrelevant instances")
print("bird measurer(assessment) with 4100: ", all_data[(all_data.event_code == 4100) & (all_data.title == "Bird Measurer (Assessment)")].shape[0])
print("mushroom sorter(assessment) with 4110: ", all_data[(all_data.event_code == 4110) & (all_data.title == "Mushroom Sorter (Assessment)")].shape[0])
print("Chest Sorter (assessment) with 4110: ", all_data[(all_data.event_code == 4110) & (all_data.title == "Chest Sorter (Assessment)")].shape[0])
print("Cart Balancer (assessment) with 4110: ", all_data[(all_data.event_code == 4110) & (all_data.title == "(Assessment)")].shape[0])

no of irrelevant instances
bird measurer(assessment) with 4100:  1
mushroom sorter(assessment) with 4110:  0
Chest Sorter (assessment) with 4110:  0
Cart Balancer (assessment) with 4110:  0


# how about other installation_ids?
- totally 2352 bird measurer/4100 rows

In [19]:
print("no of irrelevant instances")
print("bird measurer(assessment) with 4100: ", train[(train.event_code == 4100) & (train.title == "Bird Measurer (Assessment)")].shape[0])
print("mushroom sorter(assessment) with 4110: ", train[(train.event_code == 4110) & (train.title == "Mushroom Sorter (Assessment)")].shape[0])
print("Chest Sorter (assessment) with 4110: ", train[(train.event_code == 4110) & (train.title == "Chest Sorter (Assessment)")].shape[0])
print("Cart Balancer (assessment) with 4110: ", train[(train.event_code == 4110) & (train.title == "(Assessment)")].shape[0])

no of irrelevant instances
bird measurer(assessment) with 4100:  2352
mushroom sorter(assessment) with 4110:  0
Chest Sorter (assessment) with 4110:  0
Cart Balancer (assessment) with 4110:  0


# distribution of test 

In [20]:
test["installation_id"].unique()

array(['00abaee7', '01242218', '017c5718', '01a44906', '01bc6cb6',
       '02256298', '0267757a', '027e7ce5', '02a29f99', '0300c576',
       '03885368', '03ac279b', '03e33699', '048e7427', '04a7bc3f',
       '04d31500', '0500e23b', '0512bf0e', '0525589b', '05488e26',
       '05771bba', '05b82cf5', '05e17e19', '0617500d', '068ae11f',
       '0754f13b', '07749e99', '08611cc8', '08671ec7', '0889b0ae',
       '090fe325', '0937340d', '09aaaf83', '09aefe80', '0a126293',
       '0a2a77b2', '0a4c0f78', '0af94ba5', '0b24b6ac', '0b607c82',
       '0d5735f2', '0d735146', '0d7752d3', '0dd670e9', '0de6863d',
       '0e514571', '0e718764', '0ea27b66', '0f584054', '0f7116a6',
       '101999d8', '101d16f5', '108044a0', '109ad724', '10acf963',
       '1121f331', '1181ce7c', '11fa34d0', '125a3d09', '12771ee9',
       '1294d68e', '12bcbbce', '13629687', '138a2ecc', '13a0754c',
       '13bcaf23', '13cf3fc0', '13d608cb', '140087ce', '140ea7a3',
       '1423dc8f', '14cdc97f', '153f087c', '1594c19e', '15d869

In [21]:
#for i in range(test[(test["installation_id"] == "00abaee7") & (test.type == "Game")].shape[0]):#
#    print(test[(test["installation_id"] == "00abaee7") & (test.type == "Game")].iloc[i]["event_#data"])

In [22]:
test[test["installation_id"] == "50797f53"]

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
347096,4901243f,30abb60176577135,2019-09-27T22:12:11.720Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",50797f53,1,2000,0,Fireworks (Activity),Activity,TREETOPCITY
347097,beb0a7b9,30abb60176577135,2019-09-27T22:12:11.746Z,"{""description"":""Let's set off these fireworks....",50797f53,2,3010,0,Fireworks (Activity),Activity,TREETOPCITY
347098,b88f38da,30abb60176577135,2019-09-27T22:12:15.690Z,"{""description"":""Let's set off these fireworks....",50797f53,3,3110,4090,Fireworks (Activity),Activity,TREETOPCITY
347099,884228c8,30abb60176577135,2019-09-27T22:12:18.240Z,"{""coordinates"":{""x"":631,""y"":647,""stage_width"":...",50797f53,4,4070,6623,Fireworks (Activity),Activity,TREETOPCITY
347100,884228c8,30abb60176577135,2019-09-27T22:12:18.971Z,"{""coordinates"":{""x"":593,""y"":666,""stage_width"":...",50797f53,5,4070,7356,Fireworks (Activity),Activity,TREETOPCITY
347101,02a42007,30abb60176577135,2019-09-27T22:12:19.358Z,"{""rocket"":6,""coordinates"":{""x"":613,""y"":684,""st...",50797f53,6,4030,7739,Fireworks (Activity),Activity,TREETOPCITY
347102,e694a35b,30abb60176577135,2019-09-27T22:12:19.753Z,"{""rocket"":6,""height"":681,""duration"":350,""coord...",50797f53,7,4020,8089,Fireworks (Activity),Activity,TREETOPCITY
347103,884228c8,30abb60176577135,2019-09-27T22:12:19.907Z,"{""coordinates"":{""x"":799,""y"":643,""stage_width"":...",50797f53,8,4070,8306,Fireworks (Activity),Activity,TREETOPCITY
347104,884228c8,30abb60176577135,2019-09-27T22:12:20.163Z,"{""coordinates"":{""x"":763,""y"":647,""stage_width"":...",50797f53,9,4070,8556,Fireworks (Activity),Activity,TREETOPCITY
347105,884228c8,30abb60176577135,2019-09-27T22:12:20.346Z,"{""coordinates"":{""x"":735,""y"":627,""stage_width"":...",50797f53,10,4070,8739,Fireworks (Activity),Activity,TREETOPCITY


In [23]:
tmp = test[(test["installation_id"] == "08611cc8")]
tmp[(tmp.world == tmp.iloc[-1]["world"])]

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
26098,27253bdc,589fd3b020402444,2019-08-23T21:05:15.461Z,"{""event_code"": 2000, ""event_count"": 1}",08611cc8,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
26099,27253bdc,40ee12716c9a10e4,2019-08-23T21:06:11.266Z,"{""event_code"": 2000, ""event_count"": 1}",08611cc8,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
26100,27253bdc,d00ab8dcd99ef31f,2019-08-23T21:08:36.364Z,"{""event_code"": 2000, ""event_count"": 1}",08611cc8,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
26101,90d848e0,9d7c0e9474002d55,2019-08-23T21:09:31.586Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",08611cc8,1,2000,0,Cauldron Filler (Assessment),Assessment,MAGMAPEAK


In [24]:
tmp[(tmp.world == tmp.iloc[-1]["world"])].iloc[32]["event_data"]

IndexError: single positional indexer is out-of-bounds

In [25]:
print(train[(train.type=="Clip") & (train.world == "MAGMAPEAK")]["title"].unique())
print(train[(train.type=="Clip") & (train.world == "TREETOPCITY")]["title"].unique())
print(train[(train.type=="Clip") & (train.world == "CRYSTALCAVES")]["title"].unique())

['Magma Peak - Level 1' 'Slop Problem' 'Magma Peak - Level 2']
['Tree Top City - Level 1' 'Ordering Spheres' 'Costume Box' '12 Monkeys'
 'Tree Top City - Level 2' "Pirate's Tale" 'Treasure Map'
 'Tree Top City - Level 3' 'Rulers']
['Crystal Caves - Level 1' 'Balancing Act' 'Crystal Caves - Level 2'
 'Crystal Caves - Level 3' 'Lifting Heavy Things' 'Honey Cake'
 'Heavy, Heavier, Heaviest']


In [26]:
print(test[(test.type=="Clip") & (test.world == "MAGMAPEAK") ]["title"].unique())
print(test[(test.type=="Clip") & (test.world == "TREETOPCITY") ]["title"].unique())
print(test[(test.type=="Clip") & (test.world == "CRYSTALCAVES") ]["title"].unique())

['Magma Peak - Level 1' 'Magma Peak - Level 2' 'Slop Problem']
['Tree Top City - Level 1' 'Costume Box' '12 Monkeys' 'Ordering Spheres'
 'Tree Top City - Level 2' "Pirate's Tale" 'Treasure Map'
 'Tree Top City - Level 3' 'Rulers']
['Crystal Caves - Level 1' 'Balancing Act' 'Crystal Caves - Level 2'
 'Lifting Heavy Things' 'Crystal Caves - Level 3' 'Honey Cake'
 'Heavy, Heavier, Heaviest']


# ideas for feature engineering

In [27]:
train[train.type=="Assessment"].iloc[2]["event_data"]

'{"description":"Pull three mushrooms out of the ground and order them from shortest to tallest!","identifier":"Dot_PullMushrooms","media_type":"audio","total_duration":3000,"event_count":3,"game_time":37,"event_code":3010}'

In [28]:
%%time
train["x"] = train["event_data"].apply(lambda x: json.loads(x)["coordinates"]["x"] if "\"coordinates\"" in x else -999)
train["y"] = train["event_data"].apply(lambda x: json.loads(x)["coordinates"]["y"] if "\"coordinates\"" in x else -999)

CPU times: user 1min 37s, sys: 332 ms, total: 1min 37s
Wall time: 1min 37s


In [29]:
print(train[train.x != -999]["x"].min())
print(train["x"].max())

-287.0
2026.0


In [30]:
print(train[train.y != -999]["y"].min())
print(train["y"].max())

-109.0
1201.0


In [31]:
train["level"] = train["event_data"].apply(lambda x: json.loads(x)["level"] if "\"level\"" in x else -999)
train["level"].value_counts()

-999    7091003
 1        81444
 3        67843
 4        61395
 2        58710
 6        56516
 5        55239
 7        49065
 8        43514
 9        43065
 10       29559
 11       24606
 12       23929
 0        16998
 13        8922
 14        6918
 15        6040
 16        2477
 17        1976
 18        1451
 19         600
 20         422
 21         377
 24         221
 23         212
 22         180
 28         147
 27         127
 26         123
 29         120
 25         105
 42          96
 33          95
 41          86
 39          86
 40          85
 46          82
 32          81
 44          74
 31          72
 36          68
 30          67
 34          67
 37          65
 35          62
 45          60
 43          57
 38          45
 47           6
Name: level, dtype: int64

In [32]:
# version is always one, so version can't be utilized for modelling.
train["version"] = train["event_data"].apply(lambda x: json.loads(x)["version"] if "\"version\"" in x else -999)
train["version"].value_counts()

-999    7649798
1.0       79227
1          5533
Name: version, dtype: int64

In [33]:
def num_count(x): #probably id distinguishes device type
    num = [str(i) for i in range(10)]
    judge = [1 for i in x if i in num]
    return sum(judge)
test["num_in_id"] = test["installation_id"].apply(lambda x: num_count(x))
train["num_in_id"] = train["installation_id"].apply(lambda x: num_count(x))

In [34]:
train["num_in_id"].value_counts()

5    2212221
6    1875305
4    1662191
7     833186
3     748289
8     209357
2     174008
1      15712
0       4289
Name: num_in_id, dtype: int64

In [35]:
test["num_in_id"].value_counts()

5    336127
6    286348
4    261320
7    135896
3     88359
8     23306
2     15407
1      9651
Name: num_in_id, dtype: int64

In [36]:
train["game_ssion"].value_counts()

KeyError: 'game_ssion'

# check operations of get_data 

In [37]:
%%time
def encode_title(train, test, train_labels):
    train['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), train['title'], train['event_code']))
    test['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), test['title'], test['event_code']))
    all_title_event_code = list(set(train["title_event_code"].unique()).union(test["title_event_code"].unique()))
    # make a list with all the unique 'titles' from the train and test set
    list_of_user_activities = list(set(train['title'].unique()).union(set(test['title'].unique())))
    # make a list with all the unique 'event_code' from the train and test set
    list_of_event_code = list(set(train['event_code'].unique()).union(set(test['event_code'].unique())))
    list_of_event_id = list(set(train['event_id'].unique()).union(set(test['event_id'].unique())))
    # make a list with all the unique worlds from the train and test set
    list_of_worlds = list(set(train['world'].unique()).union(set(test['world'].unique())))
    # create a dictionary numerating the titles
    activities_map = dict(zip(list_of_user_activities, np.arange(len(list_of_user_activities))))
    activities_labels = dict(zip(np.arange(len(list_of_user_activities)), list_of_user_activities))
    activities_world = dict(zip(list_of_worlds, np.arange(len(list_of_worlds))))
    assess_titles = list(set(train[train['type'] == 'Assessment']['title'].value_counts().index).union(set(test[test['type'] == 'Assessment']['title'].value_counts().index)))
    # replace the text titles with the number titles from the dict
    train['title'] = train['title'].map(activities_map)
    test['title'] = test['title'].map(activities_map)
    train['world'] = train['world'].map(activities_world)
    test['world'] = test['world'].map(activities_world)
    train_labels['title'] = train_labels['title'].map(activities_map)
    win_code = dict(zip(activities_map.values(), (4100*np.ones(len(activities_map))).astype('int')))

    win_code[activities_map['Bird Measurer (Assessment)']] = 4110
    
    train['timestamp'] = pd.to_datetime(train['timestamp'])
    test['timestamp'] = pd.to_datetime(test['timestamp'])
    
    train["x"] = train["event_data"].apply(lambda x: json.loads(x)["coordinates"]["x"] if "\"coordinates\"" in x else -999)
    train["y"] = train["event_data"].apply(lambda x: json.loads(x)["coordinates"]["y"] if "\"coordinates\"" in x else -999)
    test["x"] = test["event_data"].apply(lambda x: json.loads(x)["coordinates"]["x"] if "\"coordinates\"" in x else -999)
    test["y"] = test["event_data"].apply(lambda x: json.loads(x)["coordinates"]["y"] if "\"coordinates\"" in x else -999)
        
    return train, test, train_labels, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, list_of_event_id, all_title_event_code, activities_world, list_of_worlds

# get usefull dict with maping encode
train, test, train_labels, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, list_of_event_id, all_title_event_code, activities_world, list_of_worlds = encode_title(train, test, train_labels)
categoricals = ['session_title']

CPU times: user 2min 21s, sys: 3.89 s, total: 2min 25s
Wall time: 2min 25s


In [38]:
magma_short_video = [28, 2]
magma_long_video = [22]
tree_short_video = [38, 5, 24]
tree_long_video = [43, 39, 36, 40, 31, 21]
crys_short_video =  [26, 15, 25]
crys_long_video = [18, 27, 29, 19]

def get_data(user_sample, test_set=False):
    last_activity = 0
    user_activities_count = {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
    time_spent_each_act = {actv: 0 for actv in list_of_user_activities}
    event_code_count = {eve: 0 for eve in list_of_event_code}
    user_world_count = {"world_"+str(wor) : 0 for wor in activities_world.values()} #0: None, 1:CRY, 2:TREE, 3:MEGA
    clip_videos = {"magma_intro":0, "magma_long":0, "tree_intro": 0, "tree_long": 0, "crys_intro":0, "crys_long":0}
    last_session_time_sec = 0
    all_assessments = []
    accumulated_accuracy_group = 0
    accumulated_accuracy=0
    accumulated_correct_attempts = 0 
    accumulated_uncorrect_attempts = 0 
    accumulated_actions = 0
    accumulated_stumps = 0 
    accumulated_cataps = 0
    click_counts = 0
    accumulated_clicks = 0
    audio_counts = 0
    accumulated_audio_counts = 0
    animation_counts = 0
    accumulated_anime_counts = 0
    counter = 0
    cumulative_gametime = 0
    xrange = 0
    yrange = 0 
    play_game = []
    time_first_activity = float(user_sample['timestamp'].values[0])
    durations = []
    
    for i, session in user_sample.groupby('game_session', sort=False):      
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        session_title_text = activities_labels[session_title]
        session_world = session["world"].iloc[0]
        
        # get current session time in seconds
        if session_type != 'Assessment':
            if session_type == "Clip":
                if session_title in magma_short_video:
                    clip_videos["magma_intro"] += 1
                elif session_title in magma_long_video:
                    clip_videos["magma_long"] += 1
                elif session_title in tree_short_video:
                    clip_videos["tree_intro"] += 1
                elif session_title in tree_long_video:
                    clip_videos["tree_long"] += 1
                elif session_title in crys_short_video:
                    clip_videos["crys_intro"] += 1
                elif session_title in crys_long_video:
                    clip_videos["crys_long"] += 1
                else:
                    pass
            time_spent = int(session['game_time'].iloc[-1] / 1000)
            time_spent_each_act[activities_labels[session_title]] += time_spent
            user_world_count["world_"+str(session_world)] += session.shape[0]
            
            tmp_click_counts = session["event_data"].apply(lambda x: 1 if "\"coordinates\"" in x else 0)
            click_counts += np.sum(tmp_click_counts)
            
            tmp_media = session["event_data"].apply(lambda x: json.loads(x)["media_type"] if "\"media_type\"" in x else "None").value_counts()
            if "audio" in tmp_media.index:
                audio_counts += tmp_media["audio"]
            if "animation" in tmp_media.index:
                animation_counts += tmp_media["animation"]
            
            cumulative_gametime += session.iloc[-1]["game_time"] / 1000
            
            play_game.append(session_title)
            
            if session[session.x != -999].shape[0] >= 2:
                tmp_xrange = session["x"].max() - session[session.x != -999]["x"].min()
                tmp_yrange = session["y"].max() - session[session.x != -999]["y"].min()       
            else:
                tmp_xrange = 0
                tmp_yrange = 0
            xrange = np.max([tmp_xrange, xrange])
            yrange = np.max([tmp_yrange, yrange])
            
            tmp_stump_counts = session["event_data"].apply(lambda x: np.sum(json.loads(x)["stumps"]) if "\"stumps\"" in x else 0)
            stump_counts = np.sum(tmp_stump_counts)
            accumulated_stumps += stump_counts
            #tmp_catap_counts = session["event_data"].apply(lambda x: np.sum(json.loads(x)["cataps"]) if "\"cataps\"" in x else 0)
            #catap_counts = np.sum(tmp_catap_counts)
            #accumulated_cataps += catap_counts
            
        if (session_type == 'Assessment') & (test_set or len(session)>1): # test set or session in train_label
            all_attempts = session.query(f'event_code == {win_code[session_title]}')
            true_attempts = all_attempts['event_data'].str.contains('true').sum() # 対象assessment中のtrue数
            false_attempts = all_attempts['event_data'].str.contains('false').sum() #対象assessment中のfalse数
            
            # installation_id開始から対象assessment前まで系 ------------------------
            features = user_activities_count.copy() # 対象のassessmentまでの各タイプの重複無し出現回数
            features.update(time_spent_each_act.copy()) #対象のassessmentまでの各タイトルの累積プレイ時間
            features.update(event_code_count.copy()) #対象のassessmentまでの各event_codeの累積出現回数
            features.update(user_world_count.copy()) #対象のassessmentまでの各worldの重複あり出現回数
            features.update(clip_videos.copy()) #対象のassessmentまでの各clip videoの重複あり出現回数
            features['accumulated_correct_attempts'] = accumulated_correct_attempts
            features['accumulated_uncorrect_attempts'] = accumulated_uncorrect_attempts
            accumulated_correct_attempts += true_attempts 
            accumulated_uncorrect_attempts += false_attempts
            accumulated_clicks = click_counts # 対象のassessment前までの累積click回数
            features['accumulated_clicks'] = accumulated_clicks
            accumulated_audio_counts = audio_counts # 対象のassessment前までのaudiosの累積出現回数
            features['accumulated_audios'] = accumulated_audio_counts
            accumulated_anime_counts = animation_counts # 対象のassessment前までのanimesの累積出現回数
            features['accumulated_animes'] = accumulated_anime_counts
            features['accumulated_actions'] = accumulated_actions #対象assessment前までの行数
            
            #前assessment後から次assessment前まで系 --------------------------
            # 前assess後から次assess前までの総stumps数
            features['accumulated_stumps'] = accumulated_stumps
            accumulated_stumps = 0
            #features['accumulated_cataps'] = accumulated_cataps
            #accumulated_cataps = 0
            features["total_gametime_before_assess"] = cumulative_gametime # 前assess後から次assess前までの総gameplay時間
            cumulative_gametime = 0
            features["no_playgames_before_assess"] = len(set(play_game)) # 前assess後から次assess前までのゲーム種類の数
            play_game.clear()
            features["max_xrange_before_assess"] = xrange
            features["max_yrange_before_assess"] = yrange
            xrange = 0
            yrange = 0
            features["touch_range"] = features["max_xrange_before_assess"] * features["max_yrange_before_assess"] 
            
            # 単一系 --------------------------------------------------------
            features['installation_id'] = session['installation_id'].iloc[-1]
            features['session_title'] = session_title
                                    
            # 対象assessmentの指標系 ------------------------------------------
            if durations == []: #評価対象assessmentのtimestampの期間
                features['duration_mean'] = 0
            else:
                features['duration_mean'] = np.mean(durations)
            durations.append((session.iloc[-1, 2] - session.iloc[0, 2]).seconds) 
            
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            if accuracy == 0:
                features['accuracy_group'] = 0
            elif accuracy == 1:
                features['accuracy_group'] = 3
            elif accuracy == 0.5:
                features['accuracy_group'] = 2
            else:
                features['accuracy_group'] = 1
            
            if test_set:
                all_assessments.append(features)
            elif true_attempts+false_attempts > 0:
                all_assessments.append(features)
                
            counter += 1
        
        n_of_event_codes = Counter(session['event_code']) #リスト中の出現回数を辞書形式で出力
        for key in n_of_event_codes.keys():
            event_code_count[key] += n_of_event_codes[key]
        
        accumulated_actions += len(session)
        if last_activity != session_type:
            user_activities_count[session_type] += 1
            last_activitiy = session_type
    if test_set:
        return all_assessments[-1]
    return all_assessments

In [39]:
from collections import Counter
tmp = get_data(train[train.installation_id == "0006a69f"])

In [40]:
tmp = pd.DataFrame(tmp)
tmp

,Clip,Activity,Assessment,Game,12 Monkeys,Happy Camel,Treasure Map,Tree Top City - Level 3,Sandcastle Builder (Activity),Magma Peak - Level 1,...,accumulated_stumps,total_gametime_before_assess,no_playgames_before_assess,max_xrange_before_assess,max_yrange_before_assess,touch_range,installation_id,session_title,duration_mean,accuracy_group
0,11,3,0,4,0,0,0,0,89,0,...,0,626.517,14,938.0,724.0,679112.0,0006a69f,14,0.00,3
1,14,4,1,6,0,0,0,0,89,0,...,0,376.793,6,892.0,620.0,553040.0,0006a69f,37,39.00,0
2,14,4,2,6,0,0,0,0,89,0,...,0,0.000,0,0.0,0.0,0.0,0006a69f,14,65.50,3
3,24,9,4,10,0,0,0,0,126,0,...,0,0.000,0,0.0,0.0,0.0,0006a69f,14,41.25,2
4,28,10,5,13,0,0,0,0,126,0,...,0,457.021,6,972.0,769.0,747468.0,0006a69f,37,39.20,3
